# Streaming

Streaming is an important UX consideration for LLM apps, and agents are no exception. Streaming with agents is made more complicated by the fact that it's not just tokens of the final answer that you will want to stream, but you may also want to stream back the intermediate steps an agent takes.

Our agent will use the OpenAI tools API for tool invocation, and we'll provide the agent with two tools:

1. `where_cat_is_hiding`: A tool that uses an LLM to tell us where the cat is hiding
2. `get_items`: A tool that uses an LLM to determine which items are in a given place

In this notebook, we'll see how to use `.stream` to stream action / observation pairs, and then we'll see how to use `.astream_log` to stream LLM output token by token, including from within the underlying tools.

In [6]:
from langchain import agents, hub
from langchain.prompts import ChatPromptTemplate
from langchain.tools import tool
from langchain_core.callbacks import Callbacks
from langchain_openai import ChatOpenAI

## Create the model

**Attention** 

* For older versions of langchain, we must set `streaming=True` on the LLM.

In [7]:
model = ChatOpenAI(temperature=0, streaming=True)

## Tools

We define two tools that rely on a chat model to generate output!

**Attention**

1. We invoke the model using `astream()` to force the output to stream (unfortunately for older langchain versions you should still set `streaming=True` on the model). Do not use `.ainvoke` if you need token by token streaming.
2. We attach tags to the model so that we can filter on said tags when using astream_log

In [8]:
@tool
async def where_cat_is_hiding(callbacks: Callbacks) -> str:  # <--- Accept callbacks
    """Where is the cat hiding right now?"""
    # Attach name, tags and callbacks.
    # Name
    configured_model = model.with_config(
        {"run_name": "where_cat_at", "tags": ["tool_llm"], "callbacks": callbacks}
    )
    chunks = [
        chunk
        async for chunk in configured_model.astream(
            "Think of a few places in a house where a cat likes to hangout. Then select ONE and only ONE of the places at random and write just its name.",
        )
    ]
    return "".join(chunk.content for chunk in chunks)


@tool
async def get_items(place: str, callbacks: Callbacks) -> str:  # <--- Accept callbacks
    """Use this tool to look up which items are in the given place."""
    template = ChatPromptTemplate.from_messages(
        [
            (
                "human",
                "Can you tell me what kind of items i might find in the following place: '{place}'. "
                "List at least 3 such items separating them by a comma. And include a brief description of each item..",
            )
        ]
    )
    chain = template | model.with_config(
        {
            "run_name": "Get Items LLM",
            "tags": ["tool_llm"],
            "callbacks": callbacks,
        }  # <-- Propagate callbacks
    )
    chunks = [chunk async for chunk in chain.astream({"place": place})]
    return "".join(chunk.content for chunk in chunks)

In [9]:
await where_cat_is_hiding.ainvoke({})

'Window sill'

In [10]:
await get_items.ainvoke({"place": "on a a table"})

'On a table, you might find a few common items such as:\n\n1. A book: A book is a written or printed work consisting of pages glued or sewn together along one side and bound in covers. It could be a novel, a textbook, or any other type of reading material.\n\n2. A coffee mug: A coffee mug is a cylindrical-shaped cup typically used for drinking hot beverages like coffee or tea. It usually has a handle for easy gripping and can be made of various materials such as ceramic, glass, or stainless steel.\n\n3. A vase with flowers: A vase is a decorative container, often made of glass or ceramic, used to hold flowers or other ornamental plants. It adds a touch of beauty and freshness to the surroundings, and the flowers inside can vary depending on personal preference or occasion.'

## Initialize the agent

Here, we'll initialize an OpenAI tools agent.

In [11]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
# print(prompt.messages) -- to see the prompt
tools = [get_items, where_cat_is_hiding]
agent = agents.create_openai_tools_agent(
    model.with_config({"tags": ["agent_llm"]}), tools, prompt
)
agent_executor = agents.AgentExecutor(agent=agent, tools=tools)

## Stream Intermediate Steps

We'll use `.stream` method of the AgentExecutor to stream the agent's intermediate steps.

The output from `.stream` alternates between (action, observation) pairs, finally concluding with the answer if the agent achieved its objective. 

It'll look like this:

1. actions output
2. observations output
3. actions output
4. observations output

**... (continue until goal is reached) ...**

Then, if the final goal is reached, the agent will output the **final answer**.


The contents of these outputs are summarized here:

| Output             | Contents                                                                                          |
|----------------------|------------------------------------------------------------------------------------------------------|
| **Actions**   |  <ul> <li> `actions` `AgentAction` or a subclass </li><li> `messages` chat messages corresponding to action invocation </li></ul> |
| **Observations** | <ul> <li> `steps` History of what the agent did so far, including the current action and its observation </li><li> `messages` chat message with function invocation results (aka observations) </li></ul>|
| **Final answer** | <ul> <li> `output` `AgentFinish`  </li><li> `messages` chat messages with the final output </li></ul>|

In [12]:
# Note: We use `pprint` to print only to depth 1, it makes it easier to see the output from a high level, before digging in.
import pprint

chunks = []

async for chunk in agent_executor.astream(
    {"input": "what's items are located where the cat is hiding?"}
):
    chunks.append(chunk)
    print("------")
    pprint.pprint(chunk, depth=1)

------
{'actions': [...], 'messages': [...]}
------
{'messages': [...], 'steps': [...]}
------
{'actions': [...], 'messages': [...]}
------
{'messages': [...], 'steps': [...]}
------
{'messages': [...],
 'output': 'The items located where the cat is hiding on the window sill are:\n'
           '\n'
           '1. Potted plants\n'
           '2. Decorative figurines\n'
           '3. Sun catchers'}


### Using Messages

You can access the underlying `messages` from the outputs. Using messages can be nice when working with chat applications - because everything is a message!

In [13]:
chunks[0]["actions"]

[OpenAIToolAgentAction(tool='where_cat_is_hiding', tool_input={}, log='\nInvoking: `where_cat_is_hiding` with `{}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_HIvqZ1luv8TE7flNIWHKaHsZ', 'function': {'arguments': '{}', 'name': 'where_cat_is_hiding'}, 'type': 'function'}]})], tool_call_id='call_HIvqZ1luv8TE7flNIWHKaHsZ')]

In [14]:
for chunk in chunks:
    print(chunk["messages"])

[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_HIvqZ1luv8TE7flNIWHKaHsZ', 'function': {'arguments': '{}', 'name': 'where_cat_is_hiding'}, 'type': 'function'}]})]
[FunctionMessage(content='Window sill', name='where_cat_is_hiding')]
[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8b8TvhnncfM0L5YnH6FCLqXL', 'function': {'arguments': '{\n  "place": "Window sill"\n}', 'name': 'get_items'}, 'type': 'function'}]})]
[FunctionMessage(content='In a window sill, you might find:\n\n1. Potted plants: A window sill is a perfect spot for small potted plants, such as succulents or herbs. These plants bring a touch of nature indoors and add a pop of greenery to the window area.\n\n2. Decorative figurines: Many people like to display small decorative figurines on their window sills. These can be anything from ceramic animals to miniature sculptures, adding a personal touch and enhancing the aesthetic appeal of the window.\n\n

In addition, they contain full logging information (`actions` and `steps`) which may be easier to process for rendering purposes.

### Using AgentAction/Observation

The outputs also contain richer structured information inside of `actions` and `steps`, which could be useful in some situations, but can also be harder to parse.

**Attention** `AgentFinish` is not available as part of the `streaming` method. If this is something you'd like to be added, please start a discussion on github and explain the reasoning.

In [15]:
async for chunk in agent_executor.astream(
    {"input": "what's items are located where the cat is hiding?"}
):
    # Agent Action
    if "actions" in chunk:
        for action in chunk["actions"]:
            print(f"Calling Tool: `{action.tool}` with input `{action.tool_input}`")
    # Observation
    elif "steps" in chunk:
        for step in chunk["steps"]:
            print(f"Tool Result: `{step.observation}`")
    # Final result
    elif "output" in chunk:
        print(f'Final Output: {chunk["output"]}')
    else:
        raise ValueError()
    print("---")

Calling Tool: `where_cat_is_hiding` with input `{}`
---
Tool Result: `Window sill`
---
Calling Tool: `get_items` with input `{'place': 'Window sill'}`
---
Tool Result: `In a window sill, you might find:

1. Potted plants: A window sill is a perfect spot for small potted plants, such as succulents or herbs. These plants bring a touch of nature indoors and add a pop of greenery to the window area.

2. Decorative figurines: Many people like to display small decorative figurines on their window sills. These can be anything from ceramic animals to miniature sculptures, adding a personal touch and enhancing the aesthetic appeal of the window.

3. Sun catchers: Sun catchers are often hung or placed on window sills to catch and reflect sunlight, creating beautiful patterns and colors in the room. They are typically made of glass or crystal and can add a whimsical or elegant touch to the window area.`
---
Final Output: The items located where the cat is hiding on the window sill are:

1. Potted

## Streaming Tokens & More

For some applications, you may want to stream individual LLM tokens, surface information about tool execution, or output custom messages before / after tool executions.

We will **only** stream tokens from LLMs used within tools by an agent and from no other LLMs (just to show that we can) 😮! Feel free to adapt this example to the needs of your application.
 
There are different ways in which you might be able to achieve token streaming:

1. `astream_events`: **beta** API, introduced in new langchain versions. This is the **recommended** approach.
2. [astream_log](https://python.langchain.com/docs/expression_language/interface#async-stream-intermediate-steps) API: Produces a granular log of all events that occur during execution. The log format is based on the [JSONPatch](https://jsonpatch.com/) standard. It's granular, but requirs some effort to parse.
3. `callbacks`: This can be useful if you're on older versions of LangChain and cannot upgrade. This is **NOT** recommended, as for most applications you'll need to set up a queue and send the callbacks to another worker (i.e., there's hidden complexity!). `astream_events` does this under the hood!

**ATTENTION**

* Make sure that you set the LLM to `streaming=True`
* Use async throughout (we will try to lift that restriction a bit, but for now if something isn't working use async!)

In [16]:
async for event in agent_executor.astream_events(
    {"input": "where is the cat hiding? what items are in that location?"}
):
    kind = event["event"]
    if kind == "on_chat_model_stream" and "tool_llm" in event["tags"]:
        print(event["data"]["chunk"].content, end="|")
    elif (
        kind in {"on_chat_model_start", "on_chat_model_end"}
        and "tool_llm" in event["tags"]
    ):
        print()
        print()
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Ended tool: {event['name']}")
    else:
        pass

/home/eugene/src/langchain/libs/core/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


--
Starting tool: where_cat_is_hiding with inputs: {}


|Window| sill||

Ended tool: where_cat_is_hiding
--
Starting tool: get_items with inputs: {'place': 'Window sill'}


|In| a| window| sill|,| you| might| find|:

|1|.| P|otted| plants|:| A| window| sill| is| a| perfect| spot| for| small| p|otted| plants|,| such| as| succ|ul|ents| or| herbs|.| These| plants| bring| a| touch| of| nature| indoors| and| add| a| pop| of| green|ery| to| the| window| area|.

|2|.| Decor|ative| figur|ines|:| Many| people| like| to| display| small| decorative| figur|ines| on| their| window| s|ills|.| These| can| be| anything| from| ceramic| animals| to| miniature| sculptures|,| adding| a| personal| touch| and| enhancing| the| aesthetic| appeal| of| the| window|.

|3|.| Sun| catch|ers|:| Sun| catch|ers| are| often| hung| or| placed| on| window| s|ills| to| catch| and| reflect| sunlight|,| creating| beautiful| patterns| and| colors| in| the| room|.| They| are| typically| made| of| glass| or| crystal| and| can

### Using `astream_log` (Advanced)

[astream_log](https://python.langchain.com/docs/expression_language/interface#async-stream-intermediate-steps) API: Produces a granular log of all events that occur during execution. The log format is based on the [JSONPatch](https://jsonpatch.com/) standard. It's granular so it has information, but it also requirs a bunch of effort to parse. (Please note that it is currently missing some information like "inputs" into the runnables.)

We've included sample parsing code in this example. The parsing code can be simplified or made more complex. It can also rely more on JSONPatch standard to build up the actual JSON from the underlying patches. This is entirely up to you!

Please keep in mind that `astream_events` does somthing similar under the hood for you, so if all you want is events, just use `astream_events` and enjoy life ☀️!

In [17]:
record_log_patches = [
    record_log_patch
    async for record_log_patch in agent_executor.astream_log(
        {"input": "what's items are located where the cat is hiding?"},
        include_types=["tool", "llm"],
    )
]

Below we're showing a few sample JSON patch operations. These json patch operations contain extremely granular information about all events that occurred during agent streaming.

In [18]:
record_log_patches[6]

RunLogPatch({'op': 'add',
  'path': '/streamed_output/-',
  'value': {'actions': [OpenAIToolAgentAction(tool='where_cat_is_hiding', tool_input={}, log='\nInvoking: `where_cat_is_hiding` with `{}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_CQhtjfttt3c3fXLxikbqKU4h', 'function': {'arguments': '{}', 'name': 'where_cat_is_hiding'}, 'type': 'function'}]})], tool_call_id='call_CQhtjfttt3c3fXLxikbqKU4h')],
            'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_CQhtjfttt3c3fXLxikbqKU4h', 'function': {'arguments': '{}', 'name': 'where_cat_is_hiding'}, 'type': 'function'}]})]}},
 {'op': 'replace',
  'path': '/final_output',
  'value': {'actions': [OpenAIToolAgentAction(tool='where_cat_is_hiding', tool_input={}, log='\nInvoking: `where_cat_is_hiding` with `{}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_CQhtjfttt3c3f

In [19]:
record_log_patches[10]

RunLogPatch({'op': 'add',
  'path': '/logs/where_cat_at/streamed_output_str/-',
  'value': 'Window'},
 {'op': 'add',
  'path': '/logs/where_cat_at/streamed_output/-',
  'value': AIMessageChunk(content='Window')})

In [23]:
from typing import Any, AsyncIterator, Dict, Literal, Optional, Union

from langchain_core.tracers import RunLogPatch
from typing_extensions import TypedDict


class StartEvent(TypedDict):
    """Represents a start event."""

    event_type: Literal["start"]
    start_time: float
    type: Optional[str]  # e.g., llm, tool
    name: Optional[str]  # the name of the llm or tool if assigned
    tags: Optional[Dict[str, Any]]
    metadata: Optional[Dict[str, Any]]


class EndEvent(TypedDict):
    """End event."""

    event_type: Literal["end"]
    start_time: float
    type: Optional[str]
    name: Optional[str]
    tags: Optional[Dict[str, Any]]
    metadata: Optional[Dict[str, Any]]


class StreamEvent(TypedDict):
    """Streaming event."""

    event_type: Literal["stream"]
    stream_type: Literal["str", "original"]
    type: Optional[str]
    name: Optional[str]
    tags: Optional[Dict[str, Any]]
    metadata: Optional[Dict[str, Any]]


Event = Union[StartEvent, EndEvent, StreamEvent]


async def as_event_stream(
    run_log_patches: AsyncIterator[RunLogPatch]
) -> AsyncIterator[Event]:
    """Process log patches into a list of AIMessageChunks."""
    # Info keeps track of information like tags, metadata, type, name etc.
    info: Dict[str, Any] = {}
    async for run_log_patch in run_log_patches:
        for op in run_log_patch.ops:
            if op["op"] != "add":
                continue

            path = op["path"]

            if not path.startswith("/logs/"):
                continue

            path_in_logs = path[len("/logs/") :]

            components = path_in_logs.split("/")

            if len(components) == 1:
                # It's a start event.
                name = components[0]
                value = op["value"]
                info[name] = {
                    "start_time": value["start_time"],
                    "type": value.get("type"),
                    "name": value.get("name"),
                    "tags": value.get("tags"),
                    "metadata": value.get("metadata"),
                }

                yield {
                    "event_type": "start",
                    **info[name],  # TODO(Eugene): We should make a copy here
                }
                continue
            elif len(components) == 2:
                name, kind = components
                value = op["value"]
                if kind == "final_output":
                    info["value"] = value
                    continue
                elif kind == "end_time":
                    yield {
                        "event_type": "end",
                        **info[name],  # TODO(Eugene): We should make a copy here
                    }
                    continue
                else:
                    raise ValueError(op)
            elif len(components) == 3:
                name, kind, remainder = components
                if remainder != "-":
                    raise AssertionError(components)
                if kind == "streamed_output_str":
                    value = op["value"]
                    yield {
                        "event_type": "stream",
                        "stream_type": "str",
                        "value": value,
                        **info[name],  # TODO(Eugene): We should make a copy here
                    }
                    continue
                elif kind == "streamed_output":
                    value = op["value"]
                    yield {
                        "event_type": "stream",
                        "stream_type": "original",
                        "value": value,
                        **info[name],  # TODO(Eugene): We should make a copy here
                    }
                    continue
                else:
                    raise NotImplementedError(
                        f"Parsing for op: `{op}` not implemented."
                    )
            else:
                raise NotImplementedError(f"Parsing for op: `{op}` not implemented.")

Let's materialize all events to make it convenient to work with them.

In [24]:
events = [
    event
    async for event in as_event_stream(
        agent_executor.astream_log(
            {"input": "what items are located where the cat is hiding?"},
            include_types=["tool", "llm"],
        )
    )
]
events[:3]

[{'event_type': 'start',
  'start_time': '2024-01-19T19:33:23.255+00:00',
  'type': 'llm',
  'name': 'ChatOpenAI',
  'tags': ['seq:step:3', 'agent_llm'],
  'metadata': {}},
 {'event_type': 'stream',
  'stream_type': 'str',
  'value': '',
  'start_time': '2024-01-19T19:33:23.255+00:00',
  'type': 'llm',
  'name': 'ChatOpenAI',
  'tags': ['seq:step:3', 'agent_llm'],
  'metadata': {}},
 {'event_type': 'stream',
  'stream_type': 'original',
  'value': AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_aUJ9gaedIycr6JmOrWXT5IHP', 'function': {'arguments': '', 'name': 'where_cat_is_hiding'}, 'type': 'function'}]}),
  'start_time': '2024-01-19T19:33:23.255+00:00',
  'type': 'llm',
  'name': 'ChatOpenAI',
  'tags': ['seq:step:3', 'agent_llm'],
  'metadata': {}}]

Let's use the parsed astream_log information to print stream some information about what the agent and the tools are doing!

In [25]:
from langchain_core.messages import AIMessageChunk

async for event in as_event_stream(
    agent_executor.astream_log(
        {"input": "what items are located where the cat is hiding?"},
        include_types=["tool", "llm"],
    )
):
    function_message = None
    if event["event_type"] == "start":
        tags = ", ".join(sorted(event["tags"]))
        print(f"Start >> {event['name']} ({event['type']}), tags: {tags or []}")

    if event["event_type"] == "stream" and "tool_llm" in event["tags"]:
        value = event["value"]

        if event["stream_type"] != "original":
            continue

        if not value:
            continue
        if isinstance(value, AIMessageChunk):
            # print(event['time'])
            print(value.content, end="|")
        else:
            raise NotImplementedError(type(value))

    if event["event_type"] == "end":
        print()
        print(f"End >> {event['name']} [{event['type']}]")
        print()
        function_message = None

Start >> ChatOpenAI (llm), tags: agent_llm, seq:step:3

End >> ChatOpenAI [llm]

Start >> where_cat_is_hiding (tool), tags: []
Start >> where_cat_at (llm), tags: tool_llm
|Window| sill||
End >> where_cat_at [llm]


End >> where_cat_is_hiding [tool]

Start >> ChatOpenAI (llm), tags: agent_llm, seq:step:3

End >> ChatOpenAI [llm]

Start >> get_items (tool), tags: []
Start >> Get Items LLM (llm), tags: seq:step:2, tool_llm
|In| a| window| sill|,| you| might| find|:

|1|.| P|otted| plants|:| A| window| sill| is| a| perfect| spot| for| small| p|otted| plants|,| such| as| succ|ul|ents| or| herbs|.| These| plants| bring| a| touch| of| nature| indoors| and| add| a| pop| of| green|ery| to| the| window| area|.

|2|.| Decor|ative| figur|ines|:| Many| people| like| to| display| small| decorative| figur|ines| on| their| window| s|ills|.| These| can| be| anything| from| ceramic| animals| to| miniature| sculptures|,| adding| a| personal| touch| and| enhancing| the| aesthetic| appeal| of| the| window|

### Using Callbacks (Legacy)

This can be useful if you're still on older version of LangChain and cannot upgrade. 

However, this is **NOT** a recommended approach, as for most applications you'll need to send the callbacks to another worker that can stream them to the client (i.e., there's hidden complexity to actually make it work!). 

`astream_events` does this for you under the hood!

In [28]:
from typing import TYPE_CHECKING, Any, Dict, List, Optional, Sequence, TypeVar, Union
from uuid import UUID

from langchain_core.callbacks.base import AsyncCallbackHandler
from langchain_core.messages import BaseMessage
from langchain_core.outputs import ChatGenerationChunk, GenerationChunk, LLMResult


# Here is a custom handler that will print the tokens to stdout.
# Instead of printing to stdout you can send the data elsewhere; e.g., to a streaming API response
class TokenByTokenHandler(AsyncCallbackHandler):
    def __init__(self, tags_of_interest: List[str]) -> None:
        """A custom call back handler.

        Args:
            tags_of_interest: Only LLM tokens from models with these tags will be
                              printed.
        """
        self.tags_of_interest = tags_of_interest

    async def on_chat_model_start(
        self,
        serialized: Dict[str, Any],
        messages: List[List[BaseMessage]],
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        **kwargs: Any,
    ) -> Any:
        """Run when a chat model starts running."""
        overlap_tags = self.get_overlap_tags(tags)

        if overlap_tags:
            print(",".join(overlap_tags), end=": ", flush=True)

    async def on_llm_end(
        self,
        response: LLMResult,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        tags: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> None:
        """Run when LLM ends running."""
        overlap_tags = self.get_overlap_tags(tags)

        if overlap_tags:
            # Who can argue with beauty?
            print()
            print()

    def get_overlap_tags(self, tags: Optional[List[str]]) -> List[str]:
        """Check for overlap with filtered tags."""
        if not tags:
            return []
        return sorted(set(tags or []) & set(self.tags_of_interest or []))

    async def on_llm_new_token(
        self,
        token: str,
        *,
        chunk: Optional[Union[GenerationChunk, ChatGenerationChunk]] = None,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        tags: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> None:
        """Run on new LLM token. Only available when streaming is enabled."""
        overlap_tags = self.get_overlap_tags(tags)

        if overlap_tags:
            print(token, end="|", flush=True)

In [29]:
handler = TokenByTokenHandler(tags_of_interest=["tool_llm"])

result = await agent_executor.ainvoke(
    {"input": "where is the cat hiding and what items can be found there?"},
    {"callbacks": [handler]},
)

tool_llm: |Window| sill||

tool_llm: |In| a| window| sill|,| you| might| find|:

|1|.| P|otted| plants|:| A| window| sill| is| a| perfect| spot| for| small| p|otted| plants|,| such| as| succ|ul|ents| or| herbs|.| These| plants| bring| a| touch| of| nature| indoors| and| add| a| pop| of| green|ery| to| the| window| area|.

|2|.| Decor|ative| figur|ines|:| Many| people| like| to| display| small| decorative| figur|ines| on| their| window| s|ills|.| These| can| be| anything| from| ceramic| animals| to| miniature| sculptures|,| adding| a| personal| touch| and| enhancing| the| aesthetic| appeal| of| the| window|.

|3|.| Sun| catch|ers|:| Sun| catch|ers| are| often| hung| or| placed| on| window| s|ills| to| catch| and| reflect| sunlight|,| creating| beautiful| patterns| and| colors| in| the| room|.| They| are| typically| made| of| glass| or| crystal| and| can| add| a| whims|ical| or| elegant| touch| to| the| window| area|.||

